In [1]:
pip install transformers jiwer evaluate soundfile torchcodec

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
from datasets import load_dataset, Audio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Trainer, TrainingArguments, PreTrainedTokenizerBase
import evaluate 
import re
import numpy as np
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Union

d:\SMT-5-D\Dicoding\Capstone Project\asr_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prepare Data, Tokenizer, Feature Extractor

In [3]:
# Load dataset
dataset = load_dataset("DTU54DL/common-voice",split="train")  
dataset  = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset,test_dataset  = dataset["train"],dataset["test"]

In [4]:
print(train_dataset)
print("Number of training samples: ", len(train_dataset))

Dataset({
    features: ['audio', 'sentence', 'accent'],
    num_rows: 2400
})
Number of training samples:  2400


In [5]:
print(test_dataset)
print("Number of testing samples: ", len(test_dataset))

Dataset({
    features: ['audio', 'sentence', 'accent'],
    num_rows: 601
})
Number of testing samples:  601


In [6]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

In [7]:
target_sampling_rate = processor.feature_extractor.sampling_rate 
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=target_sampling_rate))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=target_sampling_rate))

In [8]:
def normalize_text(text):
    # Hapus karakter non-alphabet (kecuali spasi dan apostrof)
    text = re.sub(r"[^A-Z' ]+", " ", text.upper())  
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [9]:
def preprocess(batch):
    audio = batch["audio"]
    batch["input_values"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["length"] = float(len(batch["input_values"]))
    batch["labels"] = processor.tokenizer(normalize_text(batch["sentence"])).input_ids
    return batch

In [10]:
train_dataset = train_dataset.map(preprocess, remove_columns=["audio", "sentence", "accent"])

In [11]:
test_dataset = test_dataset.map(preprocess, remove_columns=["audio", "sentence", "accent"])

## Training & Evaluation

In [12]:
is_cuda_available = torch.cuda.is_available()
print(f"CUDA (GPU) available: {is_cuda_available}")

if is_cuda_available:
    print(f"Device count: {torch.cuda.device_count()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA not available. Training will run on CPU (very slow).")

CUDA (GPU) available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3050 Laptop GPU


In [13]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [14]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = evaluate.load("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [16]:
training_args = TrainingArguments(
    output_dir="./wav2vec2-finetuned",
    group_by_length=False,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    eval_strategy="steps",  
    num_train_epochs=5,
    eval_steps=500,
    save_steps=500,
    logging_steps=500,
    learning_rate=5e-5,
    warmup_steps=1000,
    save_total_limit=2,
    fp16=True,
    report_to="none",
    logging_dir="./logs",
    disable_tqdm=True, 
    weight_decay=0.005,
    gradient_checkpointing=True
)

In [17]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h",
    vocab_size=len(processor.tokenizer),
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model.freeze_feature_encoder()

In [19]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,     
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class =processor
)

In [20]:
trainer.train()

d:\SMT-5-D\Dicoding\Capstone Project\asr_venv\Lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
d:\SMT-5-D\Dicoding\Capstone Project\asr_venv\Lib\site-packages\torch\utils\checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
d:\SMT-5-D\Dicoding\Capstone Project\asr_venv\Lib\site-packages\torch\backends\cudnn\__init__.py:145: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF

{'train_runtime': 889.8786, 'train_samples_per_second': 13.485, 'train_steps_per_second': 0.421, 'train_loss': 140.01579166666667, 'epoch': 5.0}


TrainOutput(global_step=375, training_loss=140.01579166666667, metrics={'train_runtime': 889.8786, 'train_samples_per_second': 13.485, 'train_steps_per_second': 0.421, 'train_loss': 140.01579166666667, 'epoch': 5.0})

In [21]:
trainer.evaluate()

d:\SMT-5-D\Dicoding\Capstone Project\asr_venv\Lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'eval_loss': 149.5011444091797, 'eval_wer': 0.22346921934312594, 'eval_runtime': 70.7608, 'eval_samples_per_second': 8.493, 'eval_steps_per_second': 1.074, 'epoch': 5.0}


{'eval_loss': 149.5011444091797,
 'eval_wer': 0.22346921934312594,
 'eval_runtime': 70.7608,
 'eval_samples_per_second': 8.493,
 'eval_steps_per_second': 1.074,
 'epoch': 5.0}